In [ ]:
from typing import List,Tuple
import matplotlib.pyplot as plt
# Класс Matrix и функции из вашей лабораторной (вставлю их без изменений для компактности)
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix
        self.rows = len(matrix)
        self.cols = len(matrix[0]) if matrix else 0

    def trans_matrix(self):
        return [[self.matrix[j][i] for j in range(self.rows)] for i in range(self.cols)]

    def __str__(self):
        return "\n".join(
            "  ".join(f"{num:.2f}" for num in row)
            for row in self.matrix
        )

In [ ]:

def gauss_forward(A: List[List[float]], b: List[List[float]], eps: float = 1e-9):
    n = len(A)
    Aug = [A[i] + b[i] for i in range(n)]
    pivot_cols = []
    row = 0
    for col in range(n):
        pivot = None
        for r in range(row, n):
            if abs(Aug[r][col]) > eps:
                pivot = r
                break
        if pivot is None:
            continue
        Aug[row], Aug[pivot] = Aug[pivot], Aug[row]
        pivot_cols.append(col)
        for r in range(row + 1, n):
            factor = Aug[r][col] / Aug[row][col]
            for c in range(col, n + 1):
                Aug[r][c] -= factor * Aug[row][c]
        row += 1
        if row == n:
            break
    return Aug, pivot_cols, row

In [ ]:

def gauss_backward(Aug: List[List[float]], pivot_cols: List[int], n: int, eps: float = 1e-9):
    x_part = [0.0] * n
    for i in reversed(range(len(pivot_cols))):
        col = pivot_cols[i]
        s = Aug[i][n]
        for j in range(col + 1, n):
            s -= Aug[i][j] * x_part[j]
        x_part[col] = s / Aug[i][col] if abs(Aug[i][col]) > eps else 0.0
    return x_part


In [ ]:
def gauss_kernel_basis(Aug: List[List[float]], pivot_cols: List[int], n: int, eps: float = 1e-9):
    free_vars = [j for j in range(n) if j not in pivot_cols]
    basis = []
    for fv in free_vars:
        vec = [0.0] * n
        vec[fv] = 1.0
        for i in reversed(range(len(pivot_cols))):
            col = pivot_cols[i]
            s = 0.0
            for j in range(col + 1, n):
                s -= Aug[i][j] * vec[j]
            vec[col] = s / Aug[i][col] if abs(Aug[i][col]) > eps else 0.0
        basis.append(vec)
    return basis

In [ ]:
def gauss_solver(A: Matrix, b: Matrix, eps: float = 1e-9):
    n = A.rows
    Aug, pivot_cols, row = gauss_forward(A.matrix, b.matrix, eps)
    for r in range(row, n):
        if all(abs(Aug[r][c]) < eps for c in range(n)) and abs(Aug[r][n]) > eps:
            raise ValueError("Система несовместна")
    x_part = gauss_backward(Aug, pivot_cols, n, eps)
    basis = gauss_kernel_basis(Aug, pivot_cols, n, eps)
    solutions = [Matrix([[val] for val in x_part])]
    for vec in basis:
        solutions.append(Matrix([[val] for val in vec]))
    return solutions

In [ ]:
def center_data(X: Matrix):
    n, m = X.rows, X.cols
    means = [sum(X.matrix[i][j] for i in range(n)) / n for j in range(m)]
    data_c = [[X.matrix[i][j] - means[j] for j in range(m)] for i in range(n)]
    return Matrix(data_c)

In [ ]:

def covariance_matrix(X_centered: Matrix):
    n, m = X_centered.rows, X_centered.cols
    X_T = X_centered.trans_matrix()
    C = [[0.0] * m for _ in range(m)]
    for i in range(m):
        for j in range(i, m):
            s = sum(X_T[i][k] * X_centered.matrix[k][j] for k in range(n))
            C[i][j] = s / (n - 1)
            C[j][i] = C[i][j]
    return Matrix(C)

In [ ]:

def find_eigenvalues(C: Matrix, tol: float = 1e-6) -> List[float]:
    m = C.rows
    A = [row.copy() for row in C.matrix]
    d = [0.0] * m
    e = [0.0] * m
    for i in range(m - 1, 0, -1):
        scale = sum(abs(A[i][k]) for k in range(i))
        if scale < tol:
            e[i] = A[i][i - 1]
            d[i] = A[i][i]
            continue
        for k in range(i):
            A[i][k] /= scale
        h = sum(A[i][k] ** 2 for k in range(i))
        f = A[i][i - 1]
        g = (-1 if f < 0 else 1) * (h ** 0.5)
        e[i] = scale * g
        h -= f * g
        A[i][i - 1] = f - g
        for j in range(i):
            e_j = sum(A[j][k] * A[i][k] for k in range(i)) / h if abs(h) > tol else 0.0
            A[j][i] = e_j
            f = sum(A[i][k] * A[j][k] for k in range(i)) / h if abs(h) > tol else 0.0
            for k in range(i):
                A[j][k] -= (e_j + f) * A[i][k]
        d[i] = h
    d[0] = A[0][0]
    e[0] = 0.0
    def sturm_count(x: float) -> int:
        count = 0
        p_prev = 1.0
        p_curr = d[0] - x
        if p_curr < 0:
            count += 1
        for i in range(1, m):
            p_next = (d[i] - x) * p_curr - (e[i] ** 2) * p_prev
            p_prev, p_curr = p_curr, p_next
            if p_curr < 0:
                count += 1
        return count
    max_e = max(abs(val) for val in e)
    lower = min(d) - max_e
    upper = max(d) + max_e
    roots = []
    for k in range(m):
        a, b = lower, upper
        while b - a > tol:
            c = 0.5 * (a + b)
            cnt = sturm_count(c)
            if cnt <= k:
                a = c
            else:
                b = c
        roots.append(0.5 * (a + b))
    return sorted(roots, reverse=True)



In [ ]:

def find_eigenvectors(C: Matrix, eigenvalues: List[float], eps: float = 1e-9) -> List[Matrix]:
    """
    Находит собственные векторы матрицы C для заданных собственных значений.

    Алгоритм:
    1. Для каждого λ решаем систему (C - λI)v = 0
    2. Находим нетривиальные решения методом Гаусса
    3. Нормируем полученные векторы

    Параметры:
        C - матрица ковариации (m x m)
        eigenvalues - список собственных значений
        eps - точность вычислений

    Возвращает:
        Список собственных векторов (каждый вектор m x 1)
    """
    m = C.rows
    eigenvectors = []

    for lambda_ in eigenvalues:
        # 1. Создаем матрицу (C - λI)
        shifted = [[C.matrix[i][j] - (lambda_ if i == j else 0.0)
                    for j in range(m)]
                   for i in range(m)]
        shifted_matrix = Matrix(shifted)

        # 2. Решаем систему (C - λI)v = 0
        try:
            # Используем ранее реализованный метод Гаусса
            solutions = gauss_solver(shifted_matrix,
                                     Matrix([[0.0] for _ in range(m)]),
                                     eps)

            # 3. Выбираем первое нетривиальное решение
            if solutions and len(solutions) > 0:
                vec = solutions[0]  # Берем первый вектор из ядра

                # 4. Нормируем вектор (делаем длину = 1)
                norm = sum(x[0] ** 2 for x in vec.matrix) ** 0.5
                if norm > eps:
                    normalized = Matrix([[x[0] / norm] for x in vec.matrix])
                    eigenvectors.append(normalized)
                else:
                    eigenvectors.append(Matrix([[0.0] for _ in range(m)]))
            else:
                eigenvectors.append(Matrix([[0.0] for _ in range(m)]))

        except ValueError:
            # Если система несовместна
            eigenvectors.append(Matrix([[0.0] for _ in range(m)]))

    return eigenvectors

In [ ]:

def explained_variance_ratio(eigenvalues, k) -> float:
    if not eigenvalues:
        raise ValueError("Список собственных значений не может быть пустым")
    if k < 0 or k > len(eigenvalues):
        raise ValueError("k должно быть в диапазоне от 0 до числа собственных значений")
    total_variance = sum(eigenvalues)
    if total_variance == 0:
        return 0.0
    explained_variance = sum(eigenvalues[:k])
    return explained_variance / total_variance


In [ ]:

def project_data(X_centered: Matrix, eigenvectors: List[Matrix], k: int) -> Matrix:
    """
    Проецирует центрированные данные на первые k главных компонент.

    Параметры:
        X_centered: центрированная матрица данных (n x m)
        eigenvectors: список собственных векторов (каждый m x 1)
        k: количество компонент для проекции

    Возвращает:
        Матрицу проекции (n x k)
    """
    # Проверка входных данных
    if k <= 0 or k > len(eigenvectors):
        raise ValueError("k должно быть положительным и не превышать количество собственных векторов")

    # Создаем матрицу V_k из первых k собственных векторов (m x k)
    V_k = [
        [eigenvectors[i].matrix[j][0] for i in range(k)]
        for j in range(X_centered.cols)
    ]

    # Умножаем X_centered (n x m) на V_k (m x k)
    projected = []
    for i in range(X_centered.rows):
        projected_row = []
        for j in range(k):
            # Скалярное произведение i-й строки данных на j-й столбец V_k
            dot_product = sum(
                X_centered.matrix[i][l] * V_k[l][j]
                for l in range(X_centered.cols)
            )
            projected_row.append(dot_product)
        projected.append(projected_row)

    return Matrix(projected)


In [ ]:
def pca(X: Matrix, k: int) -> Tuple[Matrix, float]:
    """
    Полный алгоритм PCA:
    1. Центрирование данных
    2. Расчет матрицы ковариации
    3. Нахождение собственных значений и векторов
    4. Проекция на k главных компонент
    5. Расчет доли объясненной дисперсии
    """
    # Центрирование данных
    X_centered = center_data(X)

    # Матрица ковариации
    cov_mat = covariance_matrix(X_centered)

    # Собственные значения и векторы
    eigenvalues = find_eigenvalues(cov_mat)
    eigenvectors = find_eigenvectors(cov_mat, eigenvalues)

    # Проекция данных
    X_proj = project_data(X_centered, eigenvectors, k)

    # Доля объясненной дисперсии
    explained_var = explained_variance_ratio(eigenvalues, k)

    return X_proj, explained_var

In [ ]:

# Проверка функций
print("=== Проверка функций на тестовой матрице ===")

# Тест 1: Исходная матрица из вашей лабораторной
matrix_1 = Matrix([[1, 2, 3], [2, 4, 6], [3, 6, 9]])
print("Исходная матрица:")
print(matrix_1)
print()

# Проверка center_data
centered = center_data(matrix_1)
print("Центрированная матрица:")
print(centered)
print()

# Проверка covariance_matrix
cov_matrix = covariance_matrix(centered)
print("Матрица ковариаций:")
print(cov_matrix)
print()

# Проверка find_eigenvalues
eigenvalues = find_eigenvalues(cov_matrix)
print("Собственные значения матрицы ковариаций:")
print([f"{val:.6f}" for val in eigenvalues])
print()

# Проверка explained_variance_ratio
k = 2
ratio = explained_variance_ratio(eigenvalues, k)
print(f"Доля объяснённой дисперсии для первых {k} собственных значений: {ratio:.6f}")
print()

# Проверка gauss_solver
b = Matrix([[1], [2], [3]])
try:
    solutions = gauss_solver(matrix_1, b)
    print("Решение системы Ax = b (метод Гаусса):")
    print("Частное решение:")
    print(solutions[0])
    print("Базис ядра:")
    for i in range(1, len(solutions)):
        print(f"Вектор {i}:")
        print(solutions[i])
except ValueError as e:
    print(f"Ошибка: {e}")
print()

# Тест 2: Другая матрица для разнообразия
print("=== Проверка на другой матрице ===")
matrix_2 = Matrix([[1, 2, 0], [0, 1, 1], [2, 0, 1]])
print("Исходная матрица:")
print(matrix_2)
print()

# Проверка center_data
centered_2 = center_data(matrix_2)
print("Центрированная матрица:")
print(centered_2)
print()

# Проверка covariance_matrix
cov_matrix_2 = covariance_matrix(centered_2)
print("Матрица ковариаций:")
print(cov_matrix_2)
print()

# Проверка find_eigenvalues
eigenvalues_2 = find_eigenvalues(cov_matrix_2)
print("Собственные значения матрицы ковариаций:")
print([f"{val:.6f}" for val in eigenvalues_2])
print()

# Проверка explained_variance_ratio
k = 2
ratio_2 = explained_variance_ratio(eigenvalues_2, k)
print(f"Доля объяснённой дисперсии для первых {k} собственных значений: {ratio_2:.6f}")
print()

# Проверка gauss_solver
b_2 = Matrix([[1], [1], [1]])
try:
    solutions_2 = gauss_solver(matrix_2, b_2)
    print("Решение системы Ax = b (метод Гаусса):")
    print("Частное решение:")
    print(solutions_2[0])
    print("Базис ядра:")
    for i in range(1, len(solutions_2)):
        print(f"Вектор {i}:")
        print(solutions_2[i])
except ValueError as e:
    print(f"Ошибка: {e}")


def plot_pca_projection(X_proj: Matrix) -> plt.Figure:
    """
    Визуализирует проекцию данных на первые 2 компоненты.

    Параметры:
        X_proj: матрица проекции (n x k), где k >= 2.

    Возвращает:
        Объект Figure.
    """
    if X_proj.cols < 2:
        raise ValueError("Для визуализации нужно минимум 2 компоненты")

    # Извлекаем координаты
    pc1 = [X_proj.matrix[i][0] for i in range(X_proj.rows)]
    pc2 = [X_proj.matrix[i][1] for i in range(X_proj.rows)]

    # Создаем график
    fig, ax = plt.subplots()
    ax.scatter(pc1, pc2, alpha=0.7)
    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    ax.set_title("PCA Projection")
    ax.grid(True)

    return fig


def reconstruct_data(X_proj: Matrix, eigenvectors: List[Matrix], k: int) -> Matrix:
    """
    Восстанавливает данные из проекции на главные компоненты.

    Параметры:
        X_proj: проекция данных (n x k)
        eigenvectors: собственные векторы (каждый m x 1)
        k: количество используемых компонент

    Возвращает:
        Восстановленные данные (n x m)
    """
    # Создаем матрицу V_k из первых k собственных векторов (m x k)
    V_k = [
        [eigenvectors[i].matrix[j][0] for i in range(k)]
        for j in range(len(eigenvectors[0].matrix))
    ]

    # Умножаем X_proj (n x k) на V_k^T (k x m)
    reconstructed = []
    for i in range(X_proj.rows):
        reconstructed_row = []
        for j in range(len(V_k)):
            # Скалярное произведение i-й строки проекции на j-й столбец V_k^T
            dot_product = sum(
                X_proj.matrix[i][l] * V_k[j][l]
                for l in range(k)
            )
            reconstructed_row.append(dot_product)
        reconstructed.append(reconstructed_row)

    return Matrix(reconstructed)

# 1. Загружаем данные
data = Matrix([
    [2.5, 2.4, 1.9],
    [0.5, 0.7, 0.5],
    [2.2, 2.9, 2.1],
    # ... другие данные ...
])

# 2. Применяем PCA
k = 2  # Уменьшаем до 2D
X_proj, variance = pca(data, k)

print(f"Доля объясненной дисперсии: {variance:.2%}")

# 3. Визуализация
fig = plot_pca_projection(X_proj)
plt.show()

# 4. Ошибка восстановления
X_reconstructed = reconstruct_data(X_proj,
                                 find_eigenvectors(
                                     covariance_matrix(center_data(data)),
                                     find_eigenvalues(
                                         covariance_matrix(center_data(data))
                                 )[:k]))

# 5. Расчет ошибки
mse = reconstruct_data(center_data(data), X_reconstructed)
print(f"MSE восстановления: {mse:.6f}")